In [ ]:
#problema: https://www.kaggle.com/datasets/camnugent/california-housing-prices
import pandas as pd
housing = pd.read_csv("https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv")

In [ ]:
#Analizar los datos
# Muestra las primeras 5 filas del DataFrame 'housing' para obtener una visión general de los datos
housing.head()

In [ ]:
# Muestra un resumen conciso del DataFrame 'housing', incluyendo el número de entradas, columnas, tipo de datos y valores no nulos
housing.info()

In [ ]:
# Cuenta la cantidad de apariciones de cada valor único en la columna 'ocean_proximity' del DataFrame 'housing'
housing['ocean_proximity'].value_counts()

In [ ]:
# Genera estadísticas descriptivas del DataFrame 'housing', incluyendo tanto las variables numéricas como categóricas
housing.describe(include="all")

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

# Genera histogramas para cada variable numérica en el DataFrame 'housing'
# 'bins = 50' define el número de intervalos para los histogramas
# 'figsize = (15, 10)' ajusta el tamaño de la figura a 15 por 10 pulgadas
housing.hist(bins = 50, figsize = (15, 10))

In [ ]:
# Crear conjuntos de test y train
# Stratified sampling: crear categorías que contengan muestras representativas de alguna variable que parezca importante
# Miro variables que me interesen

# Calcula la matriz de correlación del DataFrame 'housing', mostrando cómo se relacionan entre sí las variables numéricas
# Eliminar la columna 'ocean_proximity' y luego calcular la correlación
housing.drop('ocean_proximity', axis=1).corr()

In [ ]:
# 'median_income' parece representativa

# Crea una nueva categoría 'income_cat' a partir de la columna 'median_income' del DataFrame 'housing'
# 'pd.cut' segmenta los datos en intervalos especificados por 'bins'
# 'bins=[0, 1.5, 3, 4.5, 6, 16]' define los intervalos de ingreso
# 'labels=[1, 2, 3, 4, 5]' asigna una etiqueta a cada intervalo
income_cat = pd.cut(housing["median_income"], bins=[0, 1.5, 3, 4.5, 6, 16], labels=[1, 2, 3, 4, 5])


In [ ]:
# Muestra las primeras 5 filas de la nueva categoría 'income_cat' para verificar la segmentación realizada
income_cat.head()


In [ ]:
import sklearn
from sklearn.model_selection import StratifiedShuffleSplit

# Importa la clase StratifiedShuffleSplit desde sklearn.model_selection
# Crea un objeto 'split_object' de StratifiedShuffleSplit con los siguientes parámetros:
# - 'n_splits = 1' indica que se realizará una sola división estratificada
# - 'test_size = 0.2' establece el tamaño del conjunto de prueba en 20% del total
# - 'random_state = 42' fija la semilla aleatoria para asegurar reproducibilidad
split_object = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)


In [ ]:
# Genera un generador 'gen_obj' utilizando el objeto 'split_object' para dividir los datos
# 'housing' es el conjunto de datos completo
# 'income_cat' es la categoría estratificada basada en 'median_income'
gen_obj = split_object.split(housing, income_cat)


In [ ]:
# Utiliza el generador 'gen_obj' para obtener los índices de entrenamiento y prueba. 
# Los índices se refieren a las posiciones o ubicaciones de las filas dentro del DataFrame housing.
train_ind, test_ind = next(gen_obj)


In [ ]:
# Crea el conjunto de entrenamiento utilizando los índices de entrenamiento obtenidos anteriormente
strat_train_set = housing.loc[train_ind]

# Crea el conjunto de prueba utilizando los índices de prueba obtenidos anteriormente
strat_test_set = housing.loc[test_ind]


In [ ]:
# Analizar los datos más profundamente

# Crea una copia del conjunto de entrenamiento 'strat_train_set' para realizar análisis exploratorio y manipulaciones sin afectar el original
train_copy = strat_train_set.copy()

In [ ]:
# Genera un gráfico de dispersión para visualizar la distribución geográfica de los datos

train_copy.plot(kind="scatter",          # Tipo de gráfico: dispersión
                x="latitude",            # Variable en el eje x: latitud
                y="longitude",           # Variable en el eje y: longitud
                alpha=0.4,               # Transparencia de los puntos
                s=train_copy["population"]/100,  # Tamaño de los puntos basado en la población
                c="median_house_value",  # Color de los puntos basado en el valor medio de la vivienda
                cmap=plt.get_cmap("jet"),# Mapa de colores utilizado (jet)
                colorbar=True,           # Mostrar barra de colores
                figsize=(10,7)           # Tamaño de la figura (10 pulgadas de ancho, 7 pulgadas de alto)
               )


In [ ]:
# Genera una matriz de gráficos de dispersión para explorar las relaciones entre múltiples variables

pd.plotting.scatter_matrix(strat_train_set[["median_house_value", "median_income", "latitude", "total_rooms", "housing_median_age"]])


In [ ]:
# Genera una matriz de gráficos de dispersión para explorar la relación entre 'median_house_value' y 'median_income'

pd.plotting.scatter_matrix(strat_train_set[["median_house_value", "median_income"]])


In [ ]:
# Crear nuevas variables combinando información en el conjunto de entrenamiento 'train_copy'

# Variable que representa el número promedio de habitaciones por hogar
train_copy["rooms_per_household"] = train_copy["total_rooms"] / train_copy["households"]

# Variable que representa el porcentaje de dormitorios en relación al número total de habitaciones
train_copy["bedrooms_per_room"] = train_copy["total_bedrooms"] / train_copy["total_rooms"]

# Variable que representa la relación entre la población y el valor medio de la vivienda por hogar
train_copy["population_per_household"] = train_copy["population"] / train_copy["median_house_value"]


In [ ]:
#train_copy.corr()

In [ ]:
# Crea un conjunto de datos de entrenamiento eliminando la columna 'median_house_value'

train_data = strat_train_set.drop("median_house_value", axis=1)


In [ ]:
# Crea una copia de la columna 'median_house_value' del conjunto de entrenamiento como etiquetas

housing_labels = strat_train_set['median_house_value'].copy()


In [ ]:
## Transformaciones de los atributos

In [ ]:
# Manejar valores nulos utilizando SimpleImputer de sklearn

from sklearn.impute import SimpleImputer

# Crea un objeto SimpleImputer con estrategia de imputación mediana
imputer = SimpleImputer(strategy="median")

# Elimina la columna 'ocean_proximity' del conjunto de datos de entrenamiento 'train_data' para trabajar solo con datos numéricos
housing_num = train_data.drop("ocean_proximity", axis=1)

# Ajusta el imputer utilizando los datos de entrenamiento 'housing_num' (calcula la mediana en este caso por cada columna)
imputer.fit(housing_num)

# Transforma los datos numéricos 'housing_num' imputando los valores nulos con la mediana
out = imputer.transform(housing_num)

# Crea un nuevo DataFrame 'housing_tr' con los datos transformados y las columnas originales de 'housing_num'
housing_tr = pd.DataFrame(out, columns=housing_num.columns)


In [ ]:
# Genera estadísticas descriptivas del DataFrame 'housing_tr' para analizar los datos transformados

housing_tr.describe()


In [ ]:
#convertir variables no numericas a numericas- >  One Hot Encoding categories 
"""
    '<1H OCEAN' - [1,0,0,0,0]
    ‘INLAND’ - [0,1,0,0,0]
    ‘ISLAND’ - [0,0,1,0,0]
    ‘NEAR BAY’ - [0,0,0,1,0]
    ‘NEAR OCEAN' - [0,0,0,0,1]
"""

In [ ]:
# Importar la clase OneHotEncoder desde sklearn.preprocessing
from sklearn.preprocessing import OneHotEncoder

# Crear un objeto OneHotEncoder
encoder = OneHotEncoder()

# Aplicar One Hot Encoding a la columna 'ocean_proximity' del conjunto de datos de entrenamiento 'train_data'
one_hot = encoder.fit_transform(train_data[['ocean_proximity']])


In [ ]:
# El resultado de one_hot es una matriz dispersa (sparse matrix) que contiene la representación 
# one-hot encoded de la columna 'ocean_proximity'. 
print(one_hot.toarray())


In [ ]:
# Combinar atributos - armando pipelines y transformadores manuales

# Importar las clases necesarias
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# Definir la clase del transformador personalizado
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    """
    Añade atributos combinados al conjunto de datos.
    
    Parámetros:
    ----------
    add_bedrooms_per_room : bool, opcional (default=True)
        Indica si se debe añadir el atributo 'bedrooms_per_room'.
    """
    
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    def fit(self, X, y=None):
        """
        Método de ajuste del transformador.
        
        Parámetros:
        ----------
        X : array-like, shape (n_samples, n_features)
            Conjunto de datos de entrenamiento.
        y : array-like, shape (n_samples,), opcional (default=None)
            Etiquetas de entrenamiento.
        
        Retorna:
        -------
        self : CombinedAttributesAdder
            Devuelve la instancia del transformador.
        """
        return self
    
    def transform(self, X, y=None):
        """
        Método de transformación del conjunto de datos.
        
        Parámetros:
        ----------
        X : array-like, shape (n_samples, n_features)
            Conjunto de datos a transformar.
        y : array-like, shape (n_samples,), opcional (default=None)
            Etiquetas de los datos, no se utilizan en este transformador.
        
        Retorna:
        -------
        X_transformed : array-like, shape (n_samples, n_features + 2)
            Conjunto de datos transformado con los atributos adicionales.
        """
        rooms_per_household = X[:, 3] / X[:, 6]  # Calcula 'rooms_per_household'
        population_per_household = X[:, 5] / X[:, 6]  # Calcula 'population_per_household'
        
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, 4] / X[:, 3]  # Calcula 'bedrooms_per_room'
            X_transformed = np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            X_transformed = np.c_[X, rooms_per_household, population_per_household]
        
        return X_transformed

# Ejemplo de uso del transformador personalizado
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(train_data.values)


In [ ]:
# Hay que escalar los valores de las columnas
# Minmax scaling mapa features al rango de de 0 a 1
# nv = (value - min / max-min)
# Standarizacion: Mide en desviaciones estandard del valor original
# le quita peso a los outliers

# Importar la clase StandardScaler desde sklearn.preprocessing
from sklearn.preprocessing import StandardScaler

# Crear un objeto StandardScaler para escalar las características
scaler = StandardScaler()

# Aplicar el escalado a las características numéricas 'housing_num'
scaled_features = scaler.fit_transform(housing_num)

In [ ]:
# Encadenar transformadores (hacer un pipeline)
from sklearn.pipeline import Pipeline
num_pipeline = Pipeline([
          ('imputer',SimpleImputer(strategy="median")),
          ('attribs_adder',CombinedAttributesAdder()),
            ("std_scaler", StandardScaler())
                ])
housing_num_tr = num_pipeline.fit_transform(housing_num)

#Transformadores se aplican a todas las columnas

In [ ]:
# Como Componer las salidas de todas las transformaciones

# Importar la clase ColumnTransformer desde sklearn.compose y OneHotEncoder desde sklearn.preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Definir el pipeline completo utilizando ColumnTransformer
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, list(housing_num)),  # Pipeline numérico definido anteriormente
    ("cat", OneHotEncoder(), ["ocean_proximity"])  # Transformador OneHotEncoder para la columna categórica
])

# Aplicar el pipeline completo a los datos de entrenamiento 'train_data'
housing_prepared = full_pipeline.fit_transform(train_data)

In [ ]:
# Importar la clase LinearRegression desde sklearn.linear_model
from sklearn.linear_model import LinearRegression

# Crear una instancia del modelo de regresión lineal
lin_reg = LinearRegression()

# Entrenar el modelo utilizando los datos preparados 'housing_prepared' y las etiquetas 'housing_labels'
lin_reg.fit(housing_prepared, housing_labels)

# Realizar predicciones utilizando el mismo conjunto de datos de entrenamiento
predictions = lin_reg.predict(housing_prepared)

In [ ]:
# Imprimir las primeras 5 predicciones y las primeras 5 etiquetas reales
print("Predicciones:", predictions[:5])
print("Etiquetas reales:", list(housing_labels[:5]))


In [ ]:
# Medir el error

# Importar la función mean_squared_error desde sklearn.metrics -> error cuadrático medio entre las etiquetas reales y las predicciones.
from sklearn.metrics import mean_squared_error

# Calcular el (RMSE entre las etiquetas reales 'housing_labels' y las predicciones 'predictions'
lin_rmse = mean_squared_error(housing_labels, predictions, squared=False)

# Imprimir el resultado del RMSE
print("RMSE del modelo de regresión lineal:", lin_rmse)

#el error es alto probablemente el problema no se resuelva con una aproximacion lineal

In [ ]:
# Probemos con arboles de desicion

# Importar la clase DecisionTreeRegressor desde sklearn.tree
from sklearn.tree import DecisionTreeRegressor

# Crear una instancia del modelo de árbol de decisión
tree_reg = DecisionTreeRegressor()

# Entrenar el modelo utilizando los datos preparados 'housing_prepared' y las etiquetas 'housing_labels'
tree_reg.fit(housing_prepared, housing_labels)

# Realizar predicciones utilizando el mismo conjunto de datos de entrenamiento
predictions = tree_reg.predict(housing_prepared)

# Medir el error utilizando el RMSE
from sklearn.metrics import mean_squared_error
tree_rmse = mean_squared_error(housing_labels, predictions, squared=False)

# Imprimir el resultado del RMSE
print("RMSE del modelo de árbol de decisión:", tree_rmse)

In [ ]:
"""¿Hemos logrado un modelo perfecto?
Obviamente no, porque no existe tal cosa como un modelo perfecto.
Esto significa que nuestro modelo probablemente está sobreajustando.
"""

# Hagamos un testeo mas interesante: Cross validation

# Importar la función cross_val_score desde sklearn.model_selection
from sklearn.model_selection import cross_val_score

# Aplicar validación cruzada con 10 folds y usar "neg_root_mean_squared_error" como métrica de puntuación
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_root_mean_squared_error", cv=10)

# Tomar el valor absoluto de los scores (ya que cross_val_score devuelve negativos para maximizar)
scores = abs(scores)

In [ ]:
print(scores)
# veo que funciona casi peor que la regresion lineal!


In [ ]:
# veo que funciona casi peor que la regresion lineal!

# Probemos con random forest (bosques aleatorios)

# Importar la clase RandomForestRegressor desde sklearn.ensemble
from sklearn.ensemble import RandomForestRegressor

# Crear una instancia del modelo de Random Forest
forest_reg = RandomForestRegressor()

# Entrenar el modelo de Random Forest utilizando los datos preparados 'housing_prepared' y las etiquetas 'housing_labels'
forest_reg.fit(housing_prepared, housing_labels)

# Realizar predicciones utilizando el mismo conjunto de datos de entrenamiento
predictions = forest_reg.predict(housing_prepared)

# Medir el error utilizando el RMSE con los datos de entrenamiento
from sklearn.metrics import mean_squared_error
forest_rmse = mean_squared_error(housing_labels, predictions, squared=False)
print("RMSE del modelo de Random Forest (datos de entrenamiento):", forest_rmse)

# Medir el error utilizando la validación cruzada con 10 folds
scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring="neg_root_mean_squared_error", cv=10)
scores = abs(scores)
print("Scores de RMSE (validación cruzada):", scores)

In [ ]:
scores = abs(scores)
# Tenemos un modelo mas piola, antes de optimizar los parametros del modelo podemos probar si no hay otro que sea prometedor, pero en este caso nos vamos a quedar con este.
# Support Vector Machines, possibly a neural network, etc.

In [ ]:
# Fine Tuning
# probar con diferentes parametros nuestro modelo para ver si mejora
# GridSearchCV busca por todas als combinaciones podibles de parametros
# GridSearchCV(estimator, param_grid:dict, cv = None, scoring = None)

# Importar la clase GridSearchCV desde sklearn.model_selection
from sklearn.model_selection import GridSearchCV

# Crear una instancia del modelo de Random Forest
reg_forest = RandomForestRegressor()

# Definir el diccionario de parámetros que se probarán
param_grid = {
    'n_estimators': [3, 10, 30],     # Número de árboles en el bosque
    'max_features': [2, 4, 6, 8]      # Máximo número de características a considerar en cada split
}

# Configurar GridSearchCV con el modelo, los parámetros, 5 folds de validación cruzada y la métrica de RMSE negativa
grid_search = GridSearchCV(reg_forest, param_grid, cv=5, scoring="neg_root_mean_squared_error")



In [ ]:
# Ejecutar la búsqueda de hiperparámetros utilizando GridSearchCV
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
best_params = grid_search.best_params_
print("Mejores parámetros encontrados:", best_params)

In [ ]:
# Para ver los resultados de todas als combinaciones
# Obtener los resultados de la búsqueda de hiperparámetros
cvres = grid_search.cv_results_

# Iterar sobre los resultados para imprimir el score medio y los parámetros de cada combinación
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print("RMSE:", -mean_score, " | Parámetros:", params)


In [ ]:
# Evaluar el modelo en el conjunto de prueba

# Obtener X_test y y_test del conjunto de prueba
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

# Aplicar la transformación completa a X_test
X_test_prepared = full_pipeline.transform(X_test)

# Obtener el mejor modelo encontrado por GridSearchCV
final_model = grid_search.best_estimator_

# Realizar predicciones sobre X_test_prepared utilizando el modelo final
final_predictions = final_model.predict(X_test_prepared)

# Calcular el RMSE entre las etiquetas reales (y_test) y las predicciones finales
final_rmse = mean_squared_error(y_test, final_predictions, squared=False)


In [ ]:
print("RMSE final en el conjunto de prueba:", final_rmse)